In [1]:
import dijkstra3d
import numpy as np
import os
import pandas as pd
import SimpleITK as sitk
from skimage import data
import Functions as F 
import matplotlib.pyplot as plt 

In [10]:
def store_evolution_in(lst):
    """Returns a callback function to store the evolution of the level sets in
    the given list.
    """

    def _store(x):
        lst.append(np.copy(x))

    return _store
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    mr_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return itkimage, mr_scan, origin, spacing

def read_from_df(df, idx, nr): 
    position = np.array([df.loc[nr, "Agent " + str(idx) + " pos x"], df.loc[nr, "Agent " + str(idx) + " pos y"], df.loc[nr, "Agent " + str(idx) + " pos z"],])
    return position



varian_i_26, varian_l_d, varian_i_6, = [], [], []
varian_i, varian_l_d, names = [], [], []
ind = 0
weights = np.zeros((data.shape[0],3))
data = pd.read_csv(r'results\results03.csv')
nr_agents = 6
idx = F.get_best_agents(data) if nr_agents == 12 else [0, 1, 2, 3, 4, 5]
with open(r"Cmarl\src\data\filenames\testing.txt", 'r') as file: 
    filenames = file.read().split("\n")


for nr in range(data.shape[0]):
    C = read_from_df(data, idx[0], nr)
    A = read_from_df(data, idx[1], nr)
    R = read_from_df(data, idx[2], nr)
    M = read_from_df(data, idx[3], nr)
    T = read_from_df(data, idx[4], nr)
  
    #load image
    itkimage, mr_scan, origin, spacing = load_itk(filenames[nr])

    #positive (but mr is always positive)
    field = mr_scan     
    
    # path is an [N,3] numpy array i.e. a list of x,y,z coordinates
    # terminates early, default is 26 connected
    # conectivity 18 2 landmarks
    path_18 = dijkstra3d.dijkstra(field, R, T, connectivity=18, bidirectional=True) 
    
    label = mr_scan.copy()*0
    for (x,y,z) in path_18:
        label[x,y,z]=1
    sitk.WriteImage(sitk.GetImageFromArray(label), os.path.join('paths/paths_18',filenames[nr][36:])  , useCompression=True)
    
    
    #here we use 3 landmarks so there are two paths
    path_26=[]
    path = dijkstra3d.dijkstra(field, R, M, connectivity=26, bidirectional=True)
    path_26.append(path)
    label_26 = mr_scan.copy()*0
    for (x,y,z) in path:
        label_26[x,y,z]=1
    
    path = dijkstra3d.dijkstra(field, M, T, connectivity=26, bidirectional=True)
    path_26.append(path)
    for (x,y,z) in path:
        label_26[x,y,z]=1
        
    sitk.WriteImage(sitk.GetImageFromArray(label_26), os.path.join('paths/paths_26',filenames[nr][36:])  , useCompression=True)

    # connectivity 6 2 landmarks
    label_6 = mr_scan.copy()*0
    path_6 = dijkstra3d.dijkstra(field, R, T, connectivity=6,bidirectional=True)
    for (x,y,z) in path_6:
         label_6[x,y,z]=1

    sitk.WriteImage(sitk.GetImageFromArray(label_6), os.path.join('paths/paths_6',filenames[nr][36:])  , useCompression=True)



    # Compute vector with intensisties for each candidate path

    val_26=[]
    for item in path_26:
        for (x,y,z) in item:
            val_26.append(field[x,y,z])       
            
    val_18=[]

    for (x,y,z) in path_18:
        val_18.append(field[x,y,z])
    
    val_6=[]
    for (x,y,z) in path_6:
        val_6.append(field[x,y,z])
        
    val_6=np.array(val_6)
    val_18=np.array(val_18)
    val_26=np.array(val_26)
    
    # Compute comulative squared derivative
    rat_26=(np.sum(np.square(np.array(val_26[1:]-val_26[:-1], dtype='int64')))+np.sum(np.square(np.array(val_26[1:2:]-val_26[0:2:-1], dtype='int64'))))#/(len(val_26))
    rat_18=(np.sum(np.square(np.array(val_18[1:]-val_18[:-1], dtype='int64')))+np.sum(np.square(np.array(val_18[1:2:]-val_18[0:2:-1], dtype='int64'))))#/(len(val_18))
    rat_6=(np.sum(np.square(np.array(val_6[1:]-val_6[:-1], dtype='int64')))+np.sum(np.square(np.array(val_6[1:2:]-val_6[0:2:-1], dtype='int64'))))#/(len(val_6))

    print(filenames[nr])
    weights[ind,:]= [rat_6,rat_26,rat_18]
    print(np.argmin([rat_6,rat_26,rat_18]))
    ind += 1
    names.append(filenames[nr])
    
    
                


./Cmarl/src/data/images/Data_Scaled/ear1-P1051_2.nii.gz
2
./Cmarl/src/data/images/Data_Scaled/ear1-P1133_1.nii.gz
2
./Cmarl/src/data/images/Data_Scaled/ear2-P0060_1.nii.gz
2
./Cmarl/src/data/images/Data_Scaled/ear2-P0060_2.nii.gz
0
./Cmarl/src/data/images/Data_Scaled/ear2-P0407_1.nii.gz
1
./Cmarl/src/data/images/Data_Scaled/ear2-P1051_1.nii.gz
2
./Cmarl/src/data/images/Data_Scaled/ear2-P1051_2.nii.gz
1
./Cmarl/src/data/images/Data_Scaled/ear2-P1065_3.nii.gz
2
./Cmarl/src/data/images/Data_Scaled/ear2-P1094_1.nii.gz
2
./Cmarl/src/data/images/Data_good_Scaled/011_1_R.nii.gz
0
./Cmarl/src/data/images/Data_good_Scaled/020_1_L.nii.gz
0
./Cmarl/src/data/images/Data_good_Scaled/020_2_L.nii.gz
0
./Cmarl/src/data/images/Data_good_Scaled/078_1_L.nii.gz
0
./Cmarl/src/data/images/Data_good_Scaled/078_1_R.nii.gz
2
./Cmarl/src/data/images/Data_good_Scaled/084_1_R.nii.gz
2
./Cmarl/src/data/images/Data_good_Scaled/028_1_R.nii.gz
0
./Cmarl/src/data/images/Data_good_Scaled/028_2_R.nii.gz
2
./Cmarl/src/da

### Copy

In [65]:
def store_evolution_in(lst):
    """Returns a callback function to store the evolution of the level sets in
    the given list.
    """

    def _store(x):
        lst.append(np.copy(x))

    return _store
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    mr_scan = sitk.GetArrayFromImage(itkimage)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))

    return itkimage, mr_scan, origin, spacing

def read_from_df(df, idx, nr): 
    position = np.array([df.loc[nr, "Agent " + str(idx) + " pos x"], df.loc[nr, "Agent " + str(idx) + " pos y"], df.loc[nr, "Agent " + str(idx) + " pos z"],])
    return position



varian_i_26, varian_l_d, varian_i_6, = [], [], []
varian_i, varian_l_d, names = [], [], []
ind = 0
weights = np.zeros((1,3))
data = pd.read_csv(r'results\results03.csv')
nr_agents = 6
idx = F.get_best_agents(data) if nr_agents == 12 else [0, 1, 2, 3, 4, 5]
with open(r"Cmarl\src\data\filenames\testing.txt", 'r') as file: 
    filenames = file.read().split("\n")


for nr in range(10, 11):
    # C = read_from_df(data, idx[0], nr)
    # A = read_from_df(data, idx[1], nr)
    # R = read_from_df(data, idx[2], nr)
    # M = read_from_df(data, idx[3], nr)
    # T = read_from_df(data, idx[4], nr)
    C = np.array([29, 69, 39])
    A = np.array([30, 69, 36])
    R = np.array([32, 70, 30])
    M = np.array([31, 69, 37])
    T = np.array([32, 68, 44])

    #load image
    itkimage, mr_scan, origin, spacing = load_itk(filenames[nr])

    #positive (but mr is always positive)
    field = mr_scan     
    
    # path is an [N,3] numpy array i.e. a list of x,y,z coordinates
    # terminates early, default is 26 connected
    # conectivity 18 2 landmarks
    path_18 = dijkstra3d.dijkstra(field, A, C, connectivity=18, bidirectional=True) 
    
    label = mr_scan.copy()*0
    for (x,y,z) in path_18:
        label[x,y,z]=1
    sitk.WriteImage(sitk.GetImageFromArray(label), os.path.join('paths/paths_18',filenames[nr][36:])  , useCompression=True)
    
    
    #here we use 3 landmarks so there are two paths
    path_26=[]
    path = dijkstra3d.dijkstra(field, R, M, connectivity=26, bidirectional=True)
    path_26.append(path)
    label_26 = mr_scan.copy()*0
    for (x,y,z) in path:
        label_26[x,y,z]=1
    
    path = dijkstra3d.dijkstra(field, M, T, connectivity=26, bidirectional=True)
    path_26.append(path)
    for (x,y,z) in path:
        label_26[x,y,z]=1
        
    sitk.WriteImage(sitk.GetImageFromArray(label_26), os.path.join('paths/paths_26',filenames[nr][36:])  , useCompression=True)

    # connectivity 6 2 landmarks
    label_6 = mr_scan.copy()*0
    path_6 = dijkstra3d.dijkstra(field, A, C, connectivity=6,bidirectional=True)
    for (x,y,z) in path_6:
         label_6[x,y,z]=1

    sitk.WriteImage(sitk.GetImageFromArray(label_6), os.path.join('paths/paths_6',filenames[nr][36:])  , useCompression=True)



    # Compute vector with intensisties for each candidate path

    val_26=[]
    for item in path_26:
        for (x,y,z) in item:
            val_26.append(field[x,y,z])       
            
    val_18=[]

    for (x,y,z) in path_18:
        val_18.append(field[x,y,z])
    
    val_6=[]
    for (x,y,z) in path_6:
        val_6.append(field[x,y,z])
        
    val_6=np.array(val_6)
    val_18=np.array(val_18)
    val_26=np.array(val_26)
    
    # Compute comulative squared derivative
    rat_26=(np.sum(np.square(np.array(val_26[1:]-val_26[:-1], dtype='int64')))+np.sum(np.square(np.array(val_26[1:2:]-val_26[0:2:-1], dtype='int64'))))#/(len(val_26))
    rat_18=(np.sum(np.square(np.array(val_18[1:]-val_18[:-1], dtype='int64')))+np.sum(np.square(np.array(val_18[1:2:]-val_18[0:2:-1], dtype='int64'))))#/(len(val_18))
    rat_6=(np.sum(np.square(np.array(val_6[1:]-val_6[:-1], dtype='int64')))+np.sum(np.square(np.array(val_6[1:2:]-val_6[0:2:-1], dtype='int64'))))#/(len(val_6))

    print(filenames[nr])
    weights[ind,:]= [rat_6,rat_26,rat_18]
    print(np.argmin([rat_6,rat_26,rat_18]))
    ind += 1
    names.append(filenames[nr])
    
    
                


./Cmarl/src/data/images/Data_good_Scaled/020_1_L.nii.gz
2


In [66]:
Viz = F.VisualizeAngle("results/results03.csv", 6)
Viz.get_coor()
Viz.plot_angle_in_plane(10)
angles = Viz.compute_all_angles()
Rot = Viz.R

c:\Users\There\OneDrive\Dokumenter\GitHub\Bachelor-thesis-ear\Functions\angleVis.py:92: RuntimeWarning: invalid value encountered in true_divide
  n /= np.linalg.norm(n)


In [67]:
path_chorda = (Rot @ path_18.T).T 
path_ = np.concatenate([path_26[0], path_26[1]], axis = 0)

path_facial = (Rot @ path_.T).T 
xn = np.linspace(-100, 100)
m, c = np.polyfit(path_chorda[:, 0], path_chorda[:, 1], 1)
yn = np.polyval([m, c], xn)

m1, c1, = np.polyfit(path_facial[:, 0], path_facial[:, 1], 1)
yn2 = np.polyval([m1, c1], xn)


In [68]:
p1 = np.array([xn[0], yn[0]])
p2 = np.array([xn[-1], yn[-1]])
chorda_direction = p2 - p1 

p3 = np.array([xn[0], yn2[0]])
p4 = np.array([xn[-1], yn2[-1]])
facial_direction = p4 - p3

angle = np.arccos(chorda_direction @ facial_direction /(np.linalg.norm(chorda_direction) * np.linalg.norm(facial_direction))) * 180 / np.pi 

In [69]:
%matplotlib qt 
import matplotlib
matplotlib.rcParams.update({'font.size': 18})
plt.scatter(path_chorda[:, 0], path_chorda[:, 1], color = "b", label = "CTY") # A
plt.scatter(path_facial[:, 0], path_facial[:, 1], color = "r", label = "FN")
plt.axis("square")
plt.plot(xn, yn)
plt.plot(xn, yn2)
plt.legend()
plt.title("Angle = " + str(angle))
plt.xticks([])
plt.yticks([])
plt.show()

In [113]:
mu = np.mean(path_, axis = 0)
U, Sigma, V = np.linalg.svd(path - mu)
direction = V[0]

array([ 0.06068246, -0.15829528, -0.98552536])

In [63]:
for i in range(len(path_facial)): 
    print(path_facial[i, 1])

33.80766007663637
34.83399241849138
35.80757322223459
36.8586263959745
37.88495873782951
38.85522966723072
39.853531302858826
40.82711210660204
41.79738303600325
42.76765396540445
43.73792489480566
43.73792489480566
43.76595560103256
44.73953640477578
45.71311720851899
46.7114188441471
47.70972047977521
48.73274294728821
49.70301387668942
49.65026233857762
